# PATH

구글 드라이브

In [7]:
import os
import sys
from google.colab import drive

drive.mount('/content/drive')
# 지금 RNN이란 폴더에서 작업해서 RNN으로 맞춘 상태입니다! 
os.chdir('/content/drive/My Drive/RNN')

Mounted at /content/drive


In [8]:
os.getcwd()

'/content/drive/My Drive/RNN'

In [9]:
root_data = f"{os.getcwd()}/data/data_hw"
root_data

'/content/drive/My Drive/RNN/data/data_hw'

In [10]:
# 데이터 불러올 때 사용할 경로로 이후에 있을 unzip과 연동되는 내용입니다! 
path_train_data = "data/data_hw/1.Training/원천데이터/일상생활및구어체_영한_train_set.json"

path_val_data = "data/data_hw/2.Validation/원천데이터/일상생활및구어체_영한_valid_set.json"


# 사전 준비 사항

In [1]:
import torch

In [2]:
device = 'cuda' if torch.cuda.is_available() else "cpu"

In [3]:
device

'cuda'

# Data

실습에서 사용했던 영어-프랑스어와 달리 현재 사용할 데이터 셋은 영어-한국어 데이터 셋입니다. 

특히 압축을 풀어야 사용하실 수 있으니 유의하여 진행해주시길 바랍니다.

다행인 점은 용량이 그리 크지 않을겁니다!

저장 경로는 위에서 path_train_data와 path_val_data와 동일하게 맞췄습니다.

In [11]:
!unzip -qq data/data_hw/1.Training/원천데이터/TS1.zip -d data/data_hw/1.Training/원천데이터

!unzip -qq data/data_hw/2.Validation/원천데이터/VS1.zip -d data/data_hw/2.Validation/원천데이터

## 데이터 구조 확인

In [12]:
import json

In [13]:
# 경로는 앞서 PATH에서 선언했고 unzip에서도 경로를 지정했습니다. 만약 여기에 변화가 생기신다면 에러가 생길 수 있으니 유의하셔야 합니다.
with open(path_train_data, 'r') as f:
  raw_train_data = json.load(f)

with open(path_val_data, 'r') as f:
  raw_val_data = json.load(f)


In [14]:
# 현재 데이터의 구조가 {'data':~~~ }로 되어있는것을 알 수 있습니다.
raw_train_data.keys()

dict_keys(['data'])

In [15]:
# 학습 데이터 개수 확인
len(raw_train_data['data'])

1200307

지금 데이터가 너무 많아서 전처리만 해도 1,2시간 걸릴 수 있습니다. 그래서 숙제에서는 매우 일부 데이터만 이용하도록 하겠습니다.

In [16]:
raw_train_data = raw_train_data['data'][:10000]
raw_val_data = raw_val_data['data'][:1000]

In [17]:
# 샘플로 하나 확인
sample_data = raw_train_data[0]

In [18]:
# 아래의 예시를 보고 데이터를 어떻게 처리할지 잘 생각하셔야 합니다!
# 보았을 때 우리가 사용해야 하는 건 ko와 en가 되고
# 실제 데이터를 사용한다고 할 때 이 외에 필요한 데이터가 더 있을 수 있습니다.
# 모델링 과정에서는 어떻게 데이터들이 저장되어 있는지 주의해서 처리하시면 됩니다.
sample_data

{'sn': 'ECOAR1A00003',
 'data_set': '일상생활및구어체',
 'domain': '해외고객과의채팅',
 'subdomain': '숙박,음식점',
 'en_original': "I'm glad to hear that, and I hope you do consider doing business with us.",
 'en': "I'm glad to hear that, and I hope you do consider doing business with us.",
 'mt': '그 소식을 들으니 기쁩니다. 우리와 거래하는 것을 고려해 보시기 바랍니다.',
 'ko': '그 말을 들으니 기쁘고, 저희와 거래하는 것을 고려해 주셨으면 합니다.',
 'source_language': 'en',
 'target_language': 'ko',
 'word_count_ko': 10.0,
 'word_count_en': 15.0,
 'word_ratio': 0.667,
 'file_name': '해외고객과의채팅_숙박,음식점.xlsx',
 'source': '크라우드 소싱',
 'license': 'open',
 'style': '구어체',
 'included_unknown_words': False,
 'ner': None}

# Preprocess

세션 설명 코드에서는 알파벳 하나하나 사용했지만, 이번 과정에서는 토큰화를 진행하여 사용하는 과정을 Dataset에 바로 적용해보고자 합니다! 

토큰화는 앞서 word embedding 세션에서 사용한 것들을 그대로 이용하고자 합니다. 세부 커스터마이징은 자유롭습니다.

전처리 과정에서 중요한 점은, 앞의 구현 코드에서 여러 개의 for loop을 이용해서 구현했었지만, 실제 데이터를 사용한다면 매우 오래 걸리게 됩니다. 실제 데이터를 사용할 때엔 결국 얼마나 효율적으로 빠르게 정리할 수 있을지 고민해보시면서 접근하시길 바랍니다.

즉, 전처리에서 챙길 통찰은 얼마나 "효율적"으로 할 수 있는가? 입니다.

## Tokenizer

In [19]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,131 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]

In [20]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [21]:
from konlpy.tag import Okt

In [22]:
okt = Okt()

In [23]:
def CustomTokenizer(corpus):
    stop = ['Josa','Punctuation','Number']
    tokenized = []
    for i,j in okt.pos(corpus, stem=True, norm=True):
      if j in stop:
        continue
      tokenized.append(i)
    return tokenized      

In [24]:
# 한국어 처리 어떻게 되고 있는지 확인
CustomTokenizer(sample_data['ko'])

['그', '말', '들다', '기쁘다', '저희', '거래', '하다', '것', '고려', '하다', '주다', '하다']

In [25]:
# 영어도 일단 토큰화가 되고 있으니 사용해보도록 합시다
CustomTokenizer(sample_data['en'])

['I',
 'm',
 'glad',
 'to',
 'hear',
 'that',
 'and',
 'I',
 'hope',
 'you',
 'do',
 'consider',
 'doing',
 'business',
 'with',
 'us']

## 본격적인 전처리 과정

전처리릏 할 때 주의할 점은, 우리가 가지고 있는 것은 train 데이터 뿐이라는 점입니다.

토큰화를 하고, token들을 수집할 때 평가데이터(혹은 테스트 데이터)에 대한 정보는 가지고 있을 수 없습니다. 그렇기 때문에 train data를 기준으로 전처리가 진행되어야 합니다.

### Tokenization

In [26]:
from tqdm import tqdm

In [27]:
#raw_train_data에서 필요없는 정보 말고 'en', 'ko'만 가져오기
raw_train_data_en=[]
raw_train_data_ko=[]
for i in range(len(raw_train_data)):
  raw_train_data_en.append(raw_train_data[i]['en'])
  raw_train_data_ko.append(raw_train_data[i]['ko'])
print(raw_train_data_en[0])
print(raw_train_data_ko[0])

#우리는 영어문장 input 넣었을때, 한국어문장 output

I'm glad to hear that, and I hope you do consider doing business with us.
그 말을 들으니 기쁘고, 저희와 거래하는 것을 고려해 주셨으면 합니다.


In [28]:
#EOS, SOS 토큰 추가
for i,j in enumerate(raw_train_data_en):
  raw_train_data_en[i]=j+'\n'
for i,j in enumerate(raw_train_data_ko):
  raw_train_data_ko[i]='\t'+j+'\n'
print(raw_train_data_en[0:5])
print(raw_train_data_ko[0:5])

["I'm glad to hear that, and I hope you do consider doing business with us.\n", "I'm definitely thinking about it, but I have some queries to ask you.\n", "In today's world, one in every five families has either a cat, dog, or both.\n", "When you tell them, we'll take care of their child.\n", 'OK, how about for swimming?\n']
['\t그 말을 들으니 기쁘고, 저희와 거래하는 것을 고려해 주셨으면 합니다.\n', '\t확실히 생각하고 있습니다만, 몇 가지 여쭤보고 싶은 게 있어요.\n', '\t오늘날 세계 5가구 중 1가구는 고양이나 개 또는 둘 다를 키우고 있습니다.\n', '\t그들에게 말하면, 저희가 그 아이들을 돌볼 것입니다.\n', '\t좋아요, 수영은 어떤가요?\n']


In [29]:
#tokenization
# tokenization 끝난 문장 보관

src_vocab = set() #영어 단어 토큰화
tar_vocab = set() #한국어 단어 토큰화

src_seq_tr = []
tar_seq_tr = []

for i in raw_train_data_en:
  src_seq_tr.append(CustomTokenizer(i))
  for j in CustomTokenizer(i): #문장에서 단어 토큰화된거 리스트형태로!! -> 여기서 하나씩 빼와서 set에 넣어주자
    src_vocab.add(j)
for i in raw_train_data_ko:
  tar_seq_tr.append(CustomTokenizer(i))
  for j in CustomTokenizer(i):
    tar_vocab.add(j)
print(src_vocab) #영어 단어 집합
print(tar_vocab) #한국어 단어 집합

print(src_seq_tr) #토큰화 끝난 영어 문장
print(tar_seq_tr) #토큰화 끝난 한국어 문장

{'significant', 'configure', 'partake', 'exchange', 'hear', 'clear', 'rate', 'body', 'But', 'cards', 'creators', 'Turkish', 'beer', 'selves', 'goods', 'These', 'promos', 'signed', 'reputation', 'PR', 'coastal', 'cookies', 'jet', 'Python', 'booking', 'finalized', 'finances', 'respond', 'discussions', 'crowdfunding', 'Here', 'flowchart', 'bubble', 'resumes', 'observing', 'types', 'viand', 'NFT', 'valve', 'sample', 'crop', 'neat', 'product', 'boom', 'broken', 'modifications', 'personally', 'checkout', 'attempted', 'fulfilled', 'Bunsen', 'to', 'facilitate', 'guide', 'citizen', 'rounds', 'relative', 'customize', 'easiest', 'excellent', 'bodysuit', 'Plummer', 'renewable', 'offshore', 'claim', 'barrier', 'hoping', 'language', 'bulgogi', 'City', 'adverts', 'root', 'conductors', 'forwarded', 'copyright', 'properties', 'introduce', 'injunction', 'intelligence', 'unit', 'requesting', 'meal', 'channel', 'lasting', 'minimum', 'deadlines', 'stay', 'adventure', 'coding', 'sort', 'browsing', 'brochure

In [30]:
src_vocab_size=len(src_vocab)+1
tar_vocab_size=len(tar_vocab)+1
print('영어 단어 개수: ',src_vocab_size)
print('한국어 단어 개수: ',tar_vocab_size)

영어 단어 개수:  6911
한국어 단어 개수:  5802


고민해볼 점: 과연 한국어 단어집합에 한국어만 있을까요?

지금 단계에서는 시간 상 체크하진 않지만 고민해볼 부분이기도 합니다.

### token - index 정리

In [31]:
# 최종 정수 인코딩 하기 전 보기 편하기 위해 sort
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

In [32]:
# token - idx dictionary
# 여기서도 동일하게 i+1로 해줘야 padding에 사용할 token을 0으로 사용할 수 있습니다!
src_to_index = dict([word,i+1] for i,word in enumerate(src_vocab))
tar_to_index = dict([word,i+1] for i,word in enumerate(tar_vocab))
print(src_to_index)
print(tar_to_index)

{'\n': 1, ',': 2, '.': 3, 'A': 4, 'AAA': 5, 'AAA1@BBB1.com.': 6, 'AAA@BBB.com': 7, 'ABCDE': 8, 'AI': 9, 'ANRs': 10, 'ATM': 11, 'ATMs': 12, 'Absolutely': 13, 'Accidents': 14, 'According': 15, 'Actually': 16, 'Additionally': 17, 'Ads': 18, 'African': 19, 'After': 20, 'Afterwards': 21, 'Again': 22, 'Agency': 23, 'Ah': 24, 'Airport': 25, 'All': 26, 'Allow': 27, 'Almost': 28, 'Along': 29, 'Alright': 30, 'Also': 31, 'Alternatively': 32, 'Although': 33, 'Am': 34, 'America': 35, 'American': 36, 'Among': 37, 'Amsterdam': 38, 'An': 39, 'And': 40, 'Angel': 41, 'Another': 42, 'Any': 43, 'Anyone': 44, 'Anything': 45, 'Anyway': 46, 'Apart': 47, 'Apologies': 48, 'Apparently': 49, 'Appraisal': 50, 'April': 51, 'Are': 52, 'Argentina': 53, 'Around': 54, 'Articles': 55, 'Artificial': 56, 'As': 57, 'Asia': 58, 'Asian': 59, 'Aside': 60, 'Associates': 61, 'Assume': 62, 'At': 63, 'August': 64, 'Australia': 65, 'Australian': 66, 'Available': 67, 'Avatars': 68, 'Avoid': 69, 'Awesome': 70, 'B': 71, 'BBB': 72, '

### 정수 인코딩

In [33]:
#encoder 입력값
#src_seq_tr: 토큰화된 영어 문장 담겨 있음 -> 정수인코딩 해주자!! 
encoder_input=[]
for sentence in src_seq_tr:
  encoded_line=[]
  for word in sentence:
    encoded_line.append(src_to_index[word])
  encoder_input.append(torch.Tensor(encoded_line).long())

print(encoder_input[0]) #단어->정수로 바꿔줬다!! maybe 1이 의미하는거는 SOS(\n)이겠지?
print(encoder_input[1])
print(encoder_input[2])

tensor([ 329, 4034, 3239, 6351, 3389, 6280, 1017,  329, 3456, 6898, 2468, 1981,
        2482, 1520, 6832, 6587,    1])
tensor([ 329, 4034, 2278, 6305,  787, 3771, 1525,  329, 3378, 5837, 5088, 6351,
        1128, 6898,    1])
tensor([ 340, 6355, 5489, 6857, 4473, 3568, 2776, 3029, 2922, 3373, 2614,  783,
        1622, 2480, 4509, 1435,    1])


In [34]:
#decoder input
#EOS가 제외되어야함
decoder_input=[]
for sentence in tar_seq_tr:
  encoded_line=[]
  for word in sentence:
    encoded_line.append(tar_to_index[word])
  encoded_line.pop(-1)
  decoder_input.append(torch.Tensor(encoded_line).long())

print(decoder_input[0])
print(decoder_input[1])
print(decoder_input[2])

tensor([   1,  659, 1611, 1354,  769, 4183,  309, 5481,  365,  460, 5481, 4456,
        5481])
tensor([   1, 5713, 2546, 4023, 1725,  192, 3369, 2115, 3063,  368, 4023])
tensor([   1, 3468, 2653,  135, 4507,  135,  469,  267, 1412, 1330, 1058, 5065,
        4023])


In [35]:
#decode target
#SOS가 제외되어야함
decoder_target=[]
for sentence in tar_seq_tr:
  timestep=0
  encoded_line=[]
  for word in sentence:
    if timestep>0:
      encoded_line.append(tar_to_index[word])
    timestep+=1
  decoder_target.append(torch.Tensor(encoded_line).long())

print(decoder_target[0])
print(decoder_target[1])
print(decoder_target[2])

tensor([ 659, 1611, 1354,  769, 4183,  309, 5481,  365,  460, 5481, 4456, 5481,
           2])
tensor([5713, 2546, 4023, 1725,  192, 3369, 2115, 3063,  368, 4023,    2])
tensor([3468, 2653,  135, 4507,  135,  469,  267, 1412, 1330, 1058, 5065, 4023,
           2])


### Padding

In [36]:
from torch.nn.utils.rnn import pad_sequence

In [37]:
# 현재 길이가 제각각인 정수화된 문장이 있습니다. padding을 통해 길이를 맞춰봅시다
# 주의사항: 여기에 사용된 값을은 정수일까요?
batch_first = True

encoder_input_tr = pad_sequence(encoder_input,batch_first=batch_first).long()
decoder_input_tr = pad_sequence(decoder_input,batch_first=batch_first).long()
decoder_target_tr = pad_sequence(decoder_target,batch_first=batch_first).long()

In [38]:
print(encoder_input_tr.size()) #영어의 길이는 38개로 맞춰짐
print(decoder_input_tr.size()) #한국어의 길이는 33개로 맞춰짐
print(decoder_target_tr.size())

torch.Size([10000, 38])
torch.Size([10000, 33])
torch.Size([10000, 33])


## Train data

tuple type으로,  (encoder_input, decoder_input, decoder_target)의 순서를 가지게 됩니다. 이는 tuple assignment를 이용하려고 하는 것이니 자유롭게 사용하셔도 좋습니다.

In [39]:
train_data = tuple([encoder_input_tr, decoder_input_tr, decoder_target_tr])

In [40]:
train_data

(tensor([[ 329, 4034, 3239,  ...,    0,    0,    0],
         [ 329, 4034, 2278,  ...,    0,    0,    0],
         [ 340, 6355, 5489,  ...,    0,    0,    0],
         ...,
         [ 778, 1569,  987,  ...,    0,    0,    0],
         [ 329,  993, 2011,  ...,    0,    0,    0],
         [ 294,  928,  329,  ...,    0,    0,    0]]),
 tensor([[   1,  659, 1611,  ...,    0,    0,    0],
         [   1, 5713, 2546,  ...,    0,    0,    0],
         [   1, 3468, 2653,  ...,    0,    0,    0],
         ...,
         [   1, 3705, 1330,  ...,    0,    0,    0],
         [   1,  651, 5741,  ...,    0,    0,    0],
         [   1, 3145, 4183,  ...,    0,    0,    0]]),
 tensor([[ 659, 1611, 1354,  ...,    0,    0,    0],
         [5713, 2546, 4023,  ...,    0,    0,    0],
         [3468, 2653,  135,  ...,    0,    0,    0],
         ...,
         [3705, 1330,  861,  ...,    0,    0,    0],
         [ 651, 5741, 1493,  ...,    0,    0,    0],
         [3145, 4183, 5519,  ...,    0,    0,    0]])

## Eval data

지금까지 진행한 것은 학습 데이터에 대한 전처리입니다.

그런데 말입니다, 과연 평가 데이터는 학습데이터랑 동일한 형태일까요?

사용하는 단어도 동일할까요?

게다가 실제 데이터라면 문장 속에 이메일이나, 전화번호등 다른 것들로 대체해야할 게 있지 않을까요?

참으로 고민할 게 많습니다. 이 모든 것들을 고려하면 좋겠지만, 현재 과제에서는 학습 데이터의 단어 집합에는 없는데 평가 데이터에서 등장하는 단어들은 어떻게 처리할지 한번 고민해봅시다.

**핵심 TODO**:만약에 본 적이 없는 token이 나온다면 어떻게 될까요?

--> Out of Vocabulary !  =지금까지 단어 집합에 없던 아이들!<br>

이를 한번에 처리해주는 토큰으로 \<UNK\>(=unknown token) 을 사용

EX)

train data : "나는 밥을 먹는다."<br>
\>> ["나", "밥", "먹다"]<br>
\>> [\<SOS\>, 나, 밥, 먹다, \<EOS\>] <BR>
\>> {\<SOS\>:0, \<EOS\>:1, \<UNK\>: 2, 나:3, 밥:4, 먹다:5}  <bR>
\>> [0, 3, 4, 5, 1] <BR>

val data: "나는 고기를 먹는다"<br>
\>> ['나', '고기' '먹다']<BR>
\>> [\<SOS\>, 나, \<UNK\>, 먹다, \<EOS\>] <BR>
\>> [0, 3, 2, 5, 1] <BR>

In [41]:
raw_val_data_en=[]
raw_val_data_ko=[]
for i in range(len(raw_val_data)):
  raw_val_data_en.append(raw_val_data[i]['en'])
  raw_val_data_ko.append(raw_val_data[i]['ko'])
print(raw_val_data_en[0])
print(raw_val_data_ko[1])

Features include heart rate monitoring, 14-day battery life, music play control display sleep and swim tracking waterproof smart notifications.
터치스크린에는 3세대 강화유리와 지문 방지 코팅이 적용되었습니다.


In [42]:
from IPython.lib.pretty import Printable
for i,j in enumerate(raw_val_data_en):
  raw_val_data_en[i]=j+'\n'
for i,j in enumerate(raw_val_data_ko):
  raw_val_data_ko[i]='\t'+j+'\n'

print(raw_val_data_en[0:5])
print(raw_val_data_ko[0:5])

['Features include heart rate monitoring, 14-day battery life, music play control display sleep and swim tracking waterproof smart notifications.\n', 'The touch screen has 3rd generation tempered glass and anti-fingerprint coating applied.\n', 'For product inquiries, please send us an email.\n', 'Hello, BBB company AAA.\n', 'I would like to introduce the Ultimate Radiance Collection Skincare Gift Set.\n']
['\t심박수 모니터링, 14일 배터리 수명, 음악 플레이 제어 디스플레이 수면 및 수영 트래킹 방수 스마트 알림 기능이 있습니다.\n', '\t터치스크린에는 3세대 강화유리와 지문 방지 코팅이 적용되었습니다.\n', '\t제품 문의는 이메일로 주시면 감사하겠습니다.\n', '\t안녕하세요, BBB회사 AAA입니다.\n', '\t저는 얼티밋 래디언스 컬렉션 스킨케어 선물 세트 제품을 소개하려고 합니다.\n']


In [47]:
#train에서 얻은 src, tar 단어 set(src_vocab, tar_vocab)

src_seq_val = []
tar_seq_val = []

#토큰화해서 넣어줌.
for i in raw_val_data_en:
  src_seq_val.append(CustomTokenizer(i))
for i in raw_val_data_ko:
  tar_seq_val.append(CustomTokenizer(i))

#train에서 얻은 단어set에 안나온게 있다면 <UNK>로 표시하기
for i in src_seq_val:
  for index,j in enumerate(i):
    if j not in src_vocab: i[index]='<UNK>'
for i in tar_seq_val:
  for index,j in enumerate(i):
    if j not in tar_vocab: i[index]='<UNK>'

print(src_seq_val) #토큰화 끝난 영어 문장
print(tar_seq_tr) #토큰화 끝난 한국어 문장

[['<UNK>', 'include', 'heart', 'rate', 'monitoring', 'day', 'battery', 'life', 'music', 'play', 'control', 'display', 'sleep', 'and', '<UNK>', 'tracking', 'waterproof', 'smart', '<UNK>', '\n'], ['The', 'touch', 'screen', 'has', 'rd', 'generation', '<UNK>', 'glass', 'and', '<UNK>', 'fingerprint', '<UNK>', 'applied', '\n'], ['For', 'product', 'inquiries', 'please', 'send', 'us', 'an', 'email', '\n'], ['Hello', 'BBB', 'company', 'AAA', '\n'], ['I', 'would', 'like', 'to', 'introduce', 'the', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '\n'], ['<UNK>', '<UNK>', '<UNK>', 'and', 'even', '<UNK>', 'skin', '\n'], ['<UNK>', '<UNK>', '<UNK>', '<UNK>', 'ingredients', 'that', '<UNK>', 'remove', 'surface', '<UNK>', '\n'], ['It', 'enhances', 'the', 'skin', 's', 'natural', '<UNK>', 'and', 'provides', '<UNK>', '\n'], ['<UNK>', 'a', '<UNK>', 'that', '<UNK>', 'from', 'the', '<UNK>', 'of', 'the', '<UNK>', 'skin', '\n'], ['Get', '<UNK>', 'skin', 'with', 'this', 'product', '\n'], ['And', 'it', 's',

In [49]:
src_seq_val[5]

['<UNK>', '<UNK>', '<UNK>', 'and', 'even', '<UNK>', 'skin', '\n']

위의 샘플을 보면 생각보다 너무 많은 단어들이 UNK로 바뀌었습니다. 이게 과연 성능에 어떤 영향을 끼칠까요? 이를 방지할 수 있는 대책은 무엇일까요?

정답은 없습니다. 자유롭게 고민해보셔도 좋을 것 같습니다.

### UNK 토큰 추가 및 단어집합, token-index 업데이트

In [ ]:
# UNK가 추가되면서 vocab과 index도 업데이트를 해야합니다.
# 즉 지금 방식으로 처리한다면, 지금 단어 집합에는 <UNK>라는 토큰이 없습니다.
# 그래서 UNK 토큰에도 정수를 할당해줘야 합니다.
# 지금 0은 padding에서 사용하고 있으니 아래의 과정은 마지막에 <UNK>를 추가해보는 것입니다.

In [50]:
# 입력 시퀀스의 마지막 토큰과 정수인코딩값 확인
list(src_to_index.items())[-1]

('é', 6910)

In [51]:
# 현재 입력 시퀀스에 사용된 token들의 개수 확인
len(src_vocab)

6910

In [52]:
# 우리는 마지막 토큰 뒤에 UNK 토큰을 추가해봅시다
# 이 때 입력 시퀀스(src_to_index)와 출력시퀀스(tar_to_index)에 모두 사용해줘야 합니다.
src_to_index["<UNK>"] = len(src_vocab)+1
tar_to_index["<UNK>"] = len(tar_vocab)+1

In [53]:
#출력시퀀스의 단어집합 확인
print(list(tar_to_index.items())[:5])
print(list(tar_to_index.items())[-1:-5:-1])

[('\t', 1), ('\n', 2), (',', 3), ('A', 4), ('AAA', 5)]
[('<UNK>', 5802), ('힘쓰다', 5801), ('힘들다', 5800), ('힘', 5799)]


In [54]:
# 입력시퀀스의 단어집합 확인
print(list(src_to_index.items())[:5])
print(list(src_to_index.items())[-1:-5:-1])

[('\n', 1), (',', 2), ('.', 3), ('A', 4), ('AAA', 5)]
[('<UNK>', 6911), ('é', 6910), ('zoom', 6909), ('zones', 6908)]


### 정수 인코딩

In [56]:
# 평가 데이터에 대해서  정수 인코딩을 진행해주세요!
# 이 과정에서 중요한 점은 우린 학습데이터만 가지고 있으니 학습 데이터에서 사용했던 toekn-index로 인코딩을 진행해야 합니다!

#encoder 입력값
#src_seq_val: 토큰화된 영어 문장 담겨 있음 -> 정수인코딩 해주자!! 
encoder_input_val=[]
for sentence in src_seq_val:
  encoded_line=[]
  for word in sentence:
    encoded_line.append(src_to_index[word])
  encoder_input_val.append(torch.Tensor(encoded_line).long())

print(encoder_input_val[0]) 
print(encoder_input_val[1])
print(encoder_input_val[2])

tensor([6911, 3573, 3393, 5125, 4265, 2222, 1286, 3926, 4313, 4788, 2038, 2438,
        5781, 1017, 6911, 6400, 6749, 5798, 6911,    1])
tensor([ 670, 6383, 5567, 3373, 5134, 3210, 6911, 3242, 1017, 6911, 3014, 6911,
        1071,    1])
tensor([ 264, 4961, 3655, 4797, 5627, 6587, 1010, 2639,    1])


In [57]:
#decoder 입력값
decoder_input_val=[]
for sentence in tar_seq_val:
  encoded_line=[]
  for word in sentence:
    encoded_line.append(tar_to_index[word])
  encoded_line.pop(-1)
  decoder_input_val.append(torch.Tensor(encoded_line).long())

print(decoder_input_val[0]) 
print(decoder_input_val[1])
print(decoder_input_val[2])

tensor([   1, 5802, 1736, 2020, 5802, 3772, 5438, 4362, 1377, 2784, 1893, 2800,
        5196, 1985, 2861, 3172,  755, 4023])
tensor([   1, 5121, 2659, 5802, 4579, 1992, 5802, 4193, 1320])
tensor([   1, 4376, 1821, 3831, 4474,  232])


In [58]:
#decode target
#SOS가 제외되어야함
decoder_target_val=[]
for sentence in tar_seq_val:
  timestep=0
  encoded_line=[]
  for word in sentence:
    if timestep>0:
      encoded_line.append(tar_to_index[word])
    timestep+=1
  decoder_target_val.append(torch.Tensor(encoded_line).long())

print(decoder_target_val[0])
print(decoder_target_val[1])
print(decoder_target_val[2])

tensor([5802, 1736, 2020, 5802, 3772, 5438, 4362, 1377, 2784, 1893, 2800, 5196,
        1985, 2861, 3172,  755, 4023,    2])
tensor([5121, 2659, 5802, 4579, 1992, 5802, 4193, 1320,    2])
tensor([4376, 1821, 3831, 4474,  232,    2])


### Padding

In [59]:
# 주의사항: 여기에 사용된 값을은 정수일까요?
batch_first = True

encoder_input_val = pad_sequence(encoder_input_val, batch_first=batch_first).long()
decoder_input_val = pad_sequence(decoder_input_val, batch_first=batch_first).long()
decoder_target_val = pad_sequence(decoder_target_val, batch_first=batch_first).long()

In [60]:
val_data = tuple([encoder_input_val, decoder_input_val, decoder_target_val])

In [62]:
print(encoder_input_val.size()) #영어의 길이는 30개로 맞춰짐
print(decoder_input_val.size()) #한국어의 길이는 32개로 맞춰짐
print(decoder_target_val.size())

torch.Size([1000, 30])
torch.Size([1000, 32])
torch.Size([1000, 32])


# Dataset

실습코드에서 사용한 것과 동일한 구조입니다! 

사실 앞서 처리가 진행되었던 모든 과정은 Dataset에서 __get_item__ 함수 내에서 처리해서 값을 리턴할 수도 있습니다.

다만 설명을 위해 이번 과제에서는 과정을 단계별로 설명 드렸습니다. 추후에 진행하실 때엔 __get_item__에서 구현해보셔도 좋을 것 같아요!

그리고 일부 단계의 경우 임의로 처리된 것들이 있습니다.(tokenization부터 stopwords 등등) 가령 이메일, 전화번호와 같은 단어들은 \<UNK\>처럼 새로운 토큰을 이용해서 처리할 수도 있으며 목적에 따라서 다른 tokenziation을 사용할 수 있습니다.

**(참고)**

 __get_item__말고 사전에 전처리를 다한 상태로 사용할 경우 사실 사용할 모든 데이터를 메모리에 올려놓고 사용하는 것과 같습니다.(변수에 할당되어 있으니)

그런데 우리가 데이터가 매매매매매우 많아진다면, 메모리가 부족해질 수도 있지 않을까요? 그렇기 때문에 때에 따라 data를 필요할 때 전처리를 하고 반환하기도 합니다. 여기서 data가 필요할때란 결국 __get_item__을 호출할 때가 될 것입니다.

정리하자면, 꼭 모든 데이터를 한번에 다 전처리해놓도 준비해놓지 않고도, 필요할 때 마다(get_item이 호출될 때마다) 전처리해서 결과를 전달하는 방법도 있습니다.

다만 해당 과제에서는 Seq2Seq에 익숙해지는 것을 목표로 하기에 이러한 점만 알고 가셔도 충분하기도 하고, 오히려 사전에 다 처리해놓는 현재 방식이 더 효율적일 수도 있습니다. 그렇기 때문에 이 점만 알고 가셔도 좋을 것 같습니다!

In [61]:
from torch.utils.data import Dataset, DataLoader

In [65]:
class textDataset(Dataset):
  def __init__(self, data, batch_first=True):
    super(textDataset, self).__init__()
    # tuple asgginment를 활용한다면 굳이 argument로 모든 데이터를 받지 않아도 됩니다!
    enc_inp, dec_inp, dec_out = data

    # 내부에서 사용할 변수들 정리하는 것으로, 모든 데이터가 torch.LongTensor로 정리된 값들이 저장됩니다.
    self.enc_inp = enc_inp
    self.batch_first = batch_first
    self.dec_inp = dec_inp
    self.dec_out = dec_out

  def __getitem__(self, idx):
    return self.enc_inp[idx], self.dec_inp[idx], self.dec_out[idx]

  def __len__(self):
    return self.enc_inp.size(int(~self.batch_first))


In [66]:
train_dataset = textDataset(train_data, batch_first=batch_first)
val_dataset = textDataset(val_data, batch_first=batch_first)

In [67]:
val_dataset[0]

(tensor([6911, 3573, 3393, 5125, 4265, 2222, 1286, 3926, 4313, 4788, 2038, 2438,
         5781, 1017, 6911, 6400, 6749, 5798, 6911,    1,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]),
 tensor([   1, 5802, 1736, 2020, 5802, 3772, 5438, 4362, 1377, 2784, 1893, 2800,
         5196, 1985, 2861, 3172,  755, 4023,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 tensor([5802, 1736, 2020, 5802, 3772, 5438, 4362, 1377, 2784, 1893, 2800, 5196,
         1985, 2861, 3172,  755, 4023,    2,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]))

# Model

In [68]:
import torch.nn as nn
import torch.functional as F

## Encoder

본 모델에서는 LSTM를 기본 단위로 가지는 Seq2Seq 모델을 구성할 것

참고 [레퍼런스](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)

In [83]:
class Encoder(nn.Module):
  def __init__(self, in_size, hid_size, tok_size, bat_first=True, num_lay=1, bidirect=False):
    super(Encoder, self).__init__()
    self.hidden_size = hid_size
    self.embedding = nn.Embedding(tok_size, in_size)
    self.num_layers = num_lay
    self.num_directional = 2 if bidirect else 1
    self.batch_first = bat_first
    self.lstm = nn.LSTM(in_size, 
                        hid_size, 
                        num_layers = num_lay,
                        batch_first = bat_first,
                        bidirectional=bidirect)
  def forward(self, x, hidden):
    """
    x: (batch_size, seq_len)
    hidden: (batch_size, seq_len, hidden_size)
    """

    # emb : (batch_size, seq_len, input_size)
    emb = self.embedding(x)

    # out : (batch_size, seq_len, hidden_size)
    # hidden: (num_layers, batch_size, hidden_size)
    out, hidden = self.lstm(emb, hidden)
    return out, hidden

## Decoder

decoder의 hidden이 아닌 out으로 예측하고, 그 값 하나를 이용해서 다음 값 계산에 사용하기 때문에 seq_len =1이 된다.

In [84]:
class Decoder(nn.Module):
  def __init__(self, in_size, hid_size, tok_size, bat_first=True, num_lay=1, bidirect=False):
    super(Decoder, self).__init__()
    self.hidden_size = hid_size
    self.embedding = nn.Embedding(tok_size, in_size)
    self.num_layers = num_lay
    self.num_directional = 2 if bidirect else 1
    self.batch_first = bat_first
    self.lstm = nn.LSTM(in_size, 
                        hid_size, 
                        num_layers = num_lay,
                        batch_first = bat_first,
                        bidirectional=bidirect)
    self.relu =  nn.ReLU()
    self.fc1 = nn.Linear(hidden_size, tok_size)

    
  def forward(self, x, hidden):
    """
    x: (batch_size, seq_len=1)
    hidden: (num_directional*num_layers, batch_size, hidden_size)
    """

    # emb: (batch_size, seq_len=1, input_size)
    emb = self.embedding(x[:, 0]).unsqueeze(1)
    out = self.relu(emb)

    # out : (batch_size, seq_len=1, hidden_size)
    # hidden: (num_layers, batch_size, hidden_size)
    out, hidden = self.lstm(out, hidden)
    # out: (batch_size, tok_size) : batch의 element별로 token에 대해 예측값을 반환
    out = self.fc1(out.squeeze(1))    
    return out, hidden

In [85]:
def init_hidden(self, x):
  """
  레퍼런스 참고
  when batch_first=True (num_directional*num_layers, batch_size, hidden_size)
  """
  batch_size = x.size(0) if self.batch_first else x.size(1)
  h0 = torch.zeros(self.num_layers*self.num_directional, batch_size, self.hidden_size).to(device)
  c0 = torch.zeros(self.num_layers*self.num_directional, batch_size, self.hidden_size).to(device)
  return h0, c0

## Seq2Seq

In [86]:
class Seq2Seq(nn.Module):
  def __init__(self, enc, dec):
    super(Seq2Seq, self).__init__()
    assert enc.hidden_size == dec.hidden_size
    assert enc.num_layers == dec.num_layers

    self.encoder = enc
    self.decoder = dec

  def forward(self, enc_inp, dec_inp, use_teacher_force=True):
    """
    enc_inp : (batch_size, enc_seq_len)
    dec_inp : (batch_size, dec_seq_len)
    """

    # 어떤 값을 써야할까요?
    batch_size = enc_inp.size(0)
    enc_seq_len = enc_inp.size(1)
    dec_seq_len = dec_inp.size(1)
    token_size =  self.decoder.fc1.out_features


    # decoder 확률(점수) 예측값 저장
    # outputs: (batch_size, seq_len, token_size)
    outputs = torch.zeros(batch_size, dec_seq_len, token_size)

    # Step 0: Encoder의 forward에 필요한 초기 hidden, cell state 계산
    enc_init_hidden = init_hidden(self.encoder, enc_inp)

    # Step 1: Encoder를 이용하여 context 벡터 생성
    # 참고: context 벡터는 output가 아님
    _, context = self.encoder(enc_inp, enc_init_hidden)

    # Step 2: Decoder의 초기 입력값을 먼저 할당
    # 참고: 첫 입력값으로 decoder의 입력값을 이용할 수 있음
    # 주의할 사항 
    #    - (batch_size, dec_seq_len)의 차원을 맞추기 
    #    - 이렇게 하는 이유는 decoder에서 forward에 들어가야하는 input의 구조(차원)이 정해져 있기 때문
    dec_inp_t = dec_inp[:, 0].unsqueeze(-1)


    # Step 3: Decoder를 이용하여 하나하나 계산(예측)하고, 입력값 업데이트하기
    # 주의사항
    #    - Encoder의 최종 리턴한 hidden state가 context 벡터고 이게 decoder에서 init_hidden에 해당
    #    - 앞서 첫 입력값은 직접 확인했으니 다음 입력값부터 하기에 range는 1부터 시작하는 구조로?!
    dec_hidden = context
    for t in range(1, dec_seq_len):
      out, dec_hidden = self.decoder(dec_inp_t, dec_hidden)
      outputs[:, t, :] = out

      if use_teacher_force:
        # 주의사항 : 차원 맞추기 
        # 해당 경우 decoder의 입력값을 가져다가 사용해야 합
        dec_inp_t =dec_inp[:, t].unsqueeze(-1)
      else:
        # 주의사항 : 차원 맞추기 
        # 해당 경우 이번 시점의 예측값을 다음 시점의 입력값으로 해야함
        dec_inp_t = out.argmax(1, keepdim=True)

    return outputs

# Setup

In [87]:
# padding에 해당하는 token도 고려하기
enc_token_size = len(src_to_index)+1
dec_token_size = len(tar_to_index)+1

# Hyper-parameter!!
# RAM 메모리 이슈로 다운 되는 경우 때문에 줄였습니다.
input_size = 4
hidden_size = 4

In [88]:
encoder = Encoder(input_size, hidden_size, enc_token_size)
decoder = Decoder(input_size, hidden_size, dec_token_size)

model = Seq2Seq(encoder, decoder).to(device)

## 생략된 내용들

모델 생성하는 과정에서 초기값이나 등등 부수적인 부분들은 현재 과제 코드상에서 생략되었습니다. 모델 개선에 관심있으신 분들은 수행하셔도 좋습니다.

In [ ]:
# ~~~

# Train

In [89]:
def train(model, loader, optimizer, cri):
  model.train()
  loss_ep = 0

  for enc_input, dec_input, dec_target in loader:
    optimizer.zero_grad()

    # output: (batch_size, seq_len, token_size)
    output = model(enc_input.to(device), dec_input.to(device), use_teacher_force=True)
    token_size = output.size(-1)

    # 지금 [문장, 문장, ...] 구조에서 문장들을 그냥 순서대로 이어 붙인 것
    # loss 계산을 위한 조정
    # output : (batch_size * seq_len, toekn_size)
    output = output.view(-1, token_size)

    # 위와 동일한 방식으로 label 값도 조정
    # dec_target: (batch_size, seq_len)
    # target (batch_size*seq_len,)
    target = dec_target.view(-1)

    loss = cri(output, target)

    loss.backward()
    optimizer.step()

    loss_ep += loss.item()
  return loss_ep/len(loader)

# Evaluation

In [90]:
import pandas as pd

In [91]:
def val(model, loader, cri):
  # 예측값도 확인하기 위해 예측값 저장하는 값
  result = torch.Tensor()

  model.eval()
  loss_ep = 0
  with torch.no_grad():
    for enc_input, dec_input, dec_target in loader:
      # 여기서 output은 en/decoder가 아닌 seq2seq의 결과값
      # output: (batch_size, seq_len, token_size)
      output = model(enc_input.to(device), dec_input.to(device), use_teacher_force=False)
      # 평가할 땐 loss 말고 예측값도 반환하기 위해 저장
      # decode_idx = 해당 시점에서 예측한 token의 정수인코딩값. 
      # 주어진 확률(점수)값에서 최대값을 선택하기
      decode_idx = output.argmax(2).detach().cpu()
      result = torch.cat((result, decode_idx), dim=0)
  
      # loss 계산을 위한 조정
      # output : (batch_size * seq_len, toekn_size)
      token_size = output.size(-1)
      output = output.view(-1, token_size)

      # 위와 동일한 방식으로 label 값도 조정
      # dec_target: (batch_size, seq_len)
      # target (batch_size*seq_len,)
      target = dec_target.view(-1)

      loss = cri(output, target)
      loss_ep += loss.item()
  return loss_ep/len(loader), result

In [92]:
# 아래의 dictionary는 정수 인코딩 값을 넣으면 해당하는 원래 알파벳이 나옴
# padding에 사용한 0도 decode해줘야 하고 이 때 ''으로 decode해주기
index_to_src =dict((i, char) for char, i in src_to_index.items())
index_to_tar =dict((i, char) for char, i in tar_to_index.items())
index_to_tar[0] = ''
index_to_src[0] = ''

In [93]:
def decode(model, sample):
  enc_input, dec_input, dec_output = sample
  enc_input = enc_input.unsqueeze(0)
  dec_input = dec_input.unsqueeze(0)
  
  model.eval()
  with torch.no_grad():
    output = model(enc_input.to(device), dec_input.to(device), use_teacher_force=False)
    decode_idx = output.argmax(2)

  sentence_inp = pd.Series(enc_input.squeeze(0).detach().cpu().numpy()).apply(lambda x: index_to_src[x])
  sentence_out = pd.Series(decode_idx.squeeze(0).detach().cpu().numpy()).apply(lambda x: index_to_tar[x])
  sentence_act = pd.Series(dec_output.detach().cpu().numpy()).apply(lambda x: index_to_tar[x])

  print(f"입력 문장 : {' '.join(sentence_inp.tolist())}")
  print(f"실제 문장 : {' '.join(sentence_act.tolist())}")
  print(f"예측 문장 : {' '.join(sentence_out.tolist())}")

In [94]:
import time
from tqdm import tqdm
from torch import optim

In [95]:
# 학습은 아래의 코드를 이용하여 진행

train_lodaer = DataLoader(train_dataset, batch_size=1024)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)

optimizer = optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()

train_loss = []
val_loss = []

epochs = 10
for epoch in tqdm(range(epochs)):
  start = time.time()

  train_loss_ep = train(model,train_lodaer,optimizer, criterion)
  val_loss_ep, val_token = val(model, val_loader, criterion)

  train_loss.append(train_loss_ep)
  val_loss.append(val_loss_ep)
  if epoch % 2 == 0:
    print(f"Epoch : {epoch}")
    print(f"Train Loss : {train_loss_ep:.4f}")
    print(f"Val Loss : {val_loss_ep:.4f}")
    decode(model, val_dataset[100])


 10%|█         | 1/10 [01:26<12:57, 86.44s/it]

Epoch : 0
Train Loss : 8.8231
Val Loss : 8.6028
입력 문장 : I would like to introduce a <UNK> <UNK> smart <UNK> <UNK> <UNK> and temperature sensor easy to use free app product 
        
실제 문장 : 저 <UNK> <UNK> 스마트 <UNK> <UNK> <UNK> 및 온도 센서 <UNK> 사용 무료 앱 제품 소개 하다 하다 
             
예측 문장 :  그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그


 30%|███       | 3/10 [04:13<09:48, 84.12s/it]

Epoch : 2
Train Loss : 7.6009
Val Loss : 7.1772
입력 문장 : I would like to introduce a <UNK> <UNK> smart <UNK> <UNK> <UNK> and temperature sensor easy to use free app product 
        
실제 문장 : 저 <UNK> <UNK> 스마트 <UNK> <UNK> <UNK> 및 온도 센서 <UNK> 사용 무료 앱 제품 소개 하다 하다 
             
예측 문장 :  드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다 드리다


 50%|█████     | 5/10 [06:59<06:58, 83.60s/it]

...

 70%|███████   | 7/10 [09:46<04:10, 83.47s/it]

Epoch : 6
Train Loss : 3.6513
Val Loss : 3.9069
입력 문장 : I would like to introduce a <UNK> <UNK> smart <UNK> <UNK> <UNK> and temperature sensor easy to use free app product 
        
실제 문장 : 저 <UNK> <UNK> 스마트 <UNK> <UNK> <UNK> 및 온도 센서 <UNK> 사용 무료 앱 제품 소개 하다 하다 
             
예측 문장 :                                


 90%|█████████ | 9/10 [12:33<01:23, 83.49s/it]

Epoch : 8
Train Loss : 2.5526
Val Loss : 3.2281
입력 문장 : I would like to introduce a <UNK> <UNK> smart <UNK> <UNK> <UNK> and temperature sensor easy to use free app product 
        
실제 문장 : 저 <UNK> <UNK> 스마트 <UNK> <UNK> <UNK> 및 온도 센서 <UNK> 사용 무료 앱 제품 소개 하다 하다 
             
예측 문장 :                                


100%|██████████| 10/10 [13:56<00:00, 83.69s/it]


In [96]:
# 학습을 진행한 다음, 모델 백업해놓기
torch.save(model.state_dict(), 'model_hw.pt')